### 13. 텐서플로에서 데이터 적재와 전처리학
#### Loading and preprocessing data

In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version은 코랩 명령입니다.
    %tensorflow_version 2.x
    %pip install -q -U tfx
    print("패키지 호환 에러는 무시해도 괜찮습니다.")
except Exception:
    pass

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "data"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

#### 연습문제 10. 이 연습문제에서 데이터셋을 다운로드 및 분할하고 tf.data.Dataset 객체를 만들어 데이터를 적재하고 효율적으로 전처리하겠습니다. 
##### 그다음 Embedding 층을 포함한 이진 분류 모델을 만들고 훈련시킵니다.

##### a. 인터넷 영화 데이터베이스의 영화 리뷰 50,000개를 담은 영화 리뷰 데이터셋을 다운로드합니다. 이 데이터는 train과 test라는 두 개의 디렉터리로 구성되어 있습니다. 
##### 각 디렉터리에는 12,500개의 긍정 리뷰를 담은 pos 서브디렉터리와 12,500개의 부정 리뷰를 담은 neg 서브디렉터리가 있습니다. 
##### 리뷰는 각각 별도의 텍스트 파일에 저장되어 있습니다. (전처리된 BOW를 포함해) 다른 파일과 디렉터리가 있지만 이 연습문제에서는 무시합니다.

In [10]:
from pathlib import Path

DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = "aclImdb_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)
path = Path(filepath).parent / "aclImdb"
path

WindowsPath('C:/Users/jaeho/.keras/datasets/aclImdb')

In [11]:
for name, subdirs, files in os.walk(path):
    indent = len(Path(name).parts) - len(path.parts)
    print("    " * indent + Path(name).parts[-1] + os.sep)
    for index, filename in enumerate(sorted(files)):
        if index == 3:
            print("    " * (indent + 1) + "...")
            break
        print("    " * (indent + 1) + filename)

aclImdb\
    README
    imdb.vocab
    imdbEr.txt
    test\
        labeledBow.feat
        urls_neg.txt
        urls_pos.txt
        neg\
            0_2.txt
            10000_4.txt
            10001_1.txt
            ...
        pos\
            0_10.txt
            10000_7.txt
            10001_9.txt
            ...
    train\
        labeledBow.feat
        unsupBow.feat
        urls_neg.txt
        ...
        neg\
            0_3.txt
            10000_4.txt
            10001_4.txt
            ...
        pos\
            0_9.txt
            10000_8.txt
            10001_10.txt
            ...
        unsup\
            0_0.txt
            10000_0.txt
            10001_0.txt
            ...


In [12]:
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]

train_pos = review_paths(path / "train" / "pos")
train_neg = review_paths(path / "train" / "neg")
test_valid_pos = review_paths(path / "test" / "pos")
test_valid_neg = review_paths(path / "test" / "neg")

len(train_pos), len(train_neg), len(test_valid_pos), len(test_valid_neg)

(12500, 12500, 12500, 12500)

##### b. 테스트 세트를 검증 세트(15,000개)와 테스트 세트(10,000개)로 나눕니다.

In [13]:
np.random.shuffle(test_valid_pos)

test_pos = test_valid_pos[:5000]
test_neg = test_valid_neg[:5000]
valid_pos = test_valid_pos[5000:]
valid_neg = test_valid_neg[5000:]

##### c. tf.data를 사용해 각 세트에 대한 효율적인 데이터셋을 만듭니다.
##### 이 데이터셋을 메모리에 적재할 수 있으므로 파이썬 코드와 tf.data.Dataset.from_tensor_slices()를 사용해 모든 데이터를 적재합니다:

In [48]:
def imdb_dataset(filepaths_positive, filepaths_negative):
    reviews = []
    labels = []
    for filepaths, label in ((filepaths_negative, 0), (filepaths_positive, 1)):
        for filepath in filepaths:
            with open(filepath, encoding="utf-8") as review_file:  # open(file, encoding='UTF8') 
                reviews.append(review_file.read())
            labels.append(label)
    return tf.data.Dataset.from_tensor_slices(
        (tf.constant(reviews), tf.constant(labels)))

In [49]:
for X, y in imdb_dataset(train_pos, train_neg).take(3):
    print(X)
    print(y)
    print()
        
# Erorr: 'cp949' codec can't decode byte 0xe2 in position 128: illegal multibyte sequence =>  encoding='UTF8' 추가

tf.Tensor(b"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int32)

tf.Tensor(b"Airport '77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich businessman Philip Stevens (James Stewart) who is flying them & a bunch of VIP's to his estate in preparation of it being opened to the public a

In [50]:
%timeit -r1 for X, y in imdb_dataset(train_pos, train_neg).repeat(10): pass

29.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [53]:
def imdb_dataset(filepaths_positive, filepaths_negative, n_read_threads=5):
    dataset_neg = tf.data.TextLineDataset(filepaths_negative, num_parallel_reads=n_read_threads)
    dataset_neg = dataset_neg.map(lambda review: (review, 0))
    dataset_pos = tf.data.TextLineDataset(filepaths_positive, num_parallel_reads = n_read_threads)
    dataset_pos = dataset_pos.map(lambda review: (review, 1))
    return tf.data.Dataset.concatenate(dataset_pos, dataset_neg)
    

In [54]:
%timeit -r1 for X, y in imdb_dataset(train_pos, train_neg).repeat(10): pass

29.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [55]:
# 이 데이터셋을 10회 반복하는데 33초 걸립니다. 데이터셋이 RAM에 캐싱되지 않고 에포크마다 다시 로드되기 때문에 매우 느립니다. 
# repeat(10) 전에 .cache()를 추가하면 이전만큼 빨라지는 것을 확인할 수 있습니다.
%timeit -r1 for X, y in imdb_dataset(train_pos, train_neg).cache().repeat(10): pass

25 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [56]:
batch_size = 32

train_set = imdb_dataset(train_pos, train_neg).shuffle(25000).batch(batch_size).prefetch(1)
valid_set = imdb_dataset(valid_pos, valid_neg).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).batch(batch_size).prefetch(1)

##### d. 리뷰를 전처리하기 위해 TextVectorization 층을 사용한 이진 분류 모델을 만드세요. TextVectorization 층을 아직 사용할 수 없다면 (또는 도전을 좋아한다면) 사용자 전처리 층을 만들어보세요. 
##### tf.strings 패키지에 있는 함수를 사용할 수 있습니다. 예를 들어 lower()로 소문자로 만들거나 regex_replace()로 구두점을 공백으로 바꾸고 split()로 공백을 기준으로 단어를 나눌 수 있습니다. 
##### 룩업 테이블을 사용해 단어 인덱스를 출력하세요. adapt() 메서드로 미리 층을 적응시켜야 합니다.

In [57]:
# review preprocessing
def preprocess(X_batch, n_words=50):
    shape = tf.shape(X_batch)*tf.constant([1, 0]) + tf.constant([0, n_words])
    Z = tf.strings.substr(X_batch, 0, 300)
    Z = tf.strings.lower(Z)
    Z = tf.strings.regex_replace(Z, b"<br\\s*/?>", b" ")
    Z = tf.strings.regex_replace(Z, b"[^a-z]", b" ")
    Z = tf.strings.split(Z)
    return Z.to_tensor(shape=shape, default_value=b"<pad>")

X_example = tf.constant(["It's a great, great movie! I loved it.", "It was terrible, run away!!!"])
preprocess(X_example)

<tf.Tensor: shape=(2, 50), dtype=string, numpy=
array([[b'it', b's', b'a', b'great', b'great', b'movie', b'i', b'loved',
        b'it', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>'],
       [b'it', b'was', b'terrible', b'run', b'away', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'

In [58]:
# preprocessing()함수의 출력과 동일한 포맷의 데이터 샘플을 입력받는 두번째 유틸리티 함수 만듬
# 이 함수는 가장 빈번한 max_size 개수의 단어로 된 리스트 출력, 가장 흔한 단어는 패딩 토큰

from collections import Counter

def get_vocabulary(data_sample, max_size = 1000):
    preprocessed_reviews = preprocess(data_sample).numpy()
    counter= Counter()
    for words in preprocessed_reviews:
        for word in words:
            if word != b"<pad>":
                counter[word] += 1
    return [b"<pad>"] + [word for word, count in counter.most_common(max_size)]

get_vocabulary(X_example)

[b'<pad>',
 b'it',
 b'great',
 b's',
 b'a',
 b'movie',
 b'i',
 b'loved',
 b'was',
 b'terrible',
 b'run',
 b'away']

In [59]:
# TextVectorization 층 만들기 : 하이퍼파라미터를 저장하는 역할 수행
# adapt() to calculate vocab dictionary using get_vocabulary
# StaticVocabularyTable to check the words in vocab dictionary 
# call() to pad word list in each review 

In [60]:
class TextVectorization(keras.layers.Layer):
    def __init__(self, max_vocabulary_size=1000, n_oov_buckets=100, dtype=tf.string, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.max_vocabulary_size = max_vocabulary_size
        self.n_oov_buckets = n_oov_buckets

    def adapt(self, data_sample):
        self.vocab = get_vocabulary(data_sample, self.max_vocabulary_size)
        words = tf.constant(self.vocab)
        word_ids = tf.range(len(self.vocab), dtype=tf.int64)
        vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
        self.table = tf.lookup.StaticVocabularyTable(vocab_init, self.n_oov_buckets)
        
    def call(self, inputs):
        preprocessed_inputs = preprocess(inputs)
        return self.table.lookup(preprocessed_inputs)

In [61]:
text_vectorization = TextVectorization()

text_vectorization.adapt(X_example)
text_vectorization(X_example)

<tf.Tensor: shape=(2, 50), dtype=int64, numpy=
array([[ 1,  3,  4,  2,  2,  5,  6,  7,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 1,  8,  9, 10, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]], dtype=int64)>

In [62]:
# 각 리뷰는 정제되고 토큰화 되었다
# 각 단어는 어휘사전의 인텍스로 인코딩 된다, 0은 <pad> 토큰
# 또 다른 TextVectorization 층을 만들고 IMDB 훈련 세트에 적용

max_vocabulary_size = 1000
n_oov_buckets = 100

sample_review_batches = train_set.map(lambda review, label: review)
sample_reviews = np.concatenate(list(sample_review_batches.as_numpy_iterator()),
                                axis=0)

text_vectorization = TextVectorization(max_vocabulary_size, n_oov_buckets,
                                       input_shape=[])
text_vectorization.adapt(sample_reviews)

In [63]:
text_vectorization(X_example)

<tf.Tensor: shape=(2, 50), dtype=int64, numpy=
array([[  9,  14,   2,  64,  64,  12,   5, 256,   9,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  9,  13, 269, 531, 334,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]],
      dtype=int64)>

In [64]:
# 어휘 사전에서 처음 단어 10개 확인, 리뷰에서 가장 많이 등장하는 단어

text_vectorization.vocab[:10]

[b'<pad>', b'the', b'a', b'of', b'and', b'i', b'to', b'is', b'this', b'it']

In [67]:
simple_example = tf.constant([[1, 3, 1, 0, 0], [2, 2, 0, 0, 0]])
tf.reduce_sum(tf.one_hot(simple_example, 4), axis=1)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[2., 2., 0., 1.],
       [3., 0., 2., 0.]], dtype=float32)>

In [68]:
# 이제 모델을 만들기 위해 모든 단어 ID를 어떤 식으로 인코딩. 
# 한가지 방법은 BoW(bag of words). 어휘 사전에 있는 각 단어에 대해 리뷰에 단어가 등장하는 횟수를 카운트

class BagOfWords(keras.layers.Layer):
    def __init__(self, n_tokens, dtype=tf.int32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.n_tokens = n_tokens
    def call(self, inputs):
        one_hot = tf.one_hot(inputs, self.n_tokens)
        return tf.reduce_sum(one_hot, axis=1)[:, 1:]

In [69]:
bag_of_words = BagOfWords(n_tokens=4)
bag_of_words(simple_example)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2., 0., 1.],
       [0., 2., 0.]], dtype=float32)>

In [70]:
n_tokens = max_vocabulary_size + n_oov_buckets + 1 # add 1 for <pad>
bag_of_words = BagOfWords(n_tokens)

In [71]:
model = keras.models.Sequential([
    text_vectorization,
    bag_of_words,
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid"),
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
782/782 [==============================] - 14s 12ms/step - loss: 0.5422 - accuracy: 0.7147 - val_loss: 0.5144 - val_accuracy: 0.7387
Epoch 2/5
782/782 [==============================] - 8s 8ms/step - loss: 0.4702 - accuracy: 0.7721 - val_loss: 0.5064 - val_accuracy: 0.7448
Epoch 3/5
782/782 [==============================] - 8s 8ms/step - loss: 0.4202 - accuracy: 0.8044 - val_loss: 0.5186 - val_accuracy: 0.7415
Epoch 4/5
782/782 [==============================] - 8s 8ms/step - loss: 0.3512 - accuracy: 0.8530 - val_loss: 0.5434 - val_accuracy: 0.7365
Epoch 5/5
782/782 [==============================] - 8s 8ms/step - loss: 0.2681 - accuracy: 0.9000 - val_loss: 0.5791 - val_accuracy: 0.7350


### e. Embedding 층을 추가하고 단어 개수의 제곱근을 곱하여 리뷰마다 평균 임베딩을 계산하세요(16장 참조). 
### 이제 스케일이 조정된 이 평균 임베딩을 모델의 다음 부분으로 전달할 수 있습니다.

#### 각 리뷰의 평균 임베딩을 계산하고 리뷰에 있는 단어 개수의 제곱근을 곱하기 위해 간단한 함수를 정의합니다. 
#### 각 문장에 대해서 이 함수는 $M \times \sqrt N$을 계산합니다. 여기에서 $M$은 (패딩 토큰을 제외하고) 문장에 있는 모든 단어 임베딩의 평균입니다. 
#### $N$은 (패딩 토큰을 제외한) 문장에 있는 단어의 개수입니다. $M$을 $\dfrac{S}{N}$로 다시 쓸 수 있습니다. 
#### 여기에서 $S$는 모든 단어 임베딩의 합입니다(패딩 토큰은 0 벡터이므로 합에서는 패딩 토큰을 포함했는지 여부가 문제가 안됩니다). 
#### 따라서 이 함수는 $M \times \sqrt N = \dfrac{S}{N} \times \sqrt N = \dfrac{S}{\sqrt N \times \sqrt N} \times \sqrt N= \dfrac{S}{\sqrt N}$를 반환해야 합니다.

In [73]:
def compute_mean_embedding(inputs):
    not_pad = tf.math.count_nonzero(inputs, axis = -1)
    n_words = tf.math.count_nonzero(not_pad, axis = -1, keepdims = True)
    sqrt_n_words = tf.math.sqrt(tf.cast(n_words, tf.float32))
    return tf.reduce_sum(inputs, axis=1) / sqrt_n_words

another_example = tf.constant([[[1., 2., 3], [4., 5., 0.], [0., 0., 0.]],
                              [[6., 0., 0.], [0., 0., 0.], [0., 0., 0.]]])
compute_mean_embedding(another_example)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[3.535534 , 4.9497476, 2.1213205],
       [6.       , 0.       , 0.       ]], dtype=float32)>

In [74]:
#결과가 올바른지 확인
# 첫번깨 리뷰에는 2개의 단어가 있다 (마지막 토큰은 <pad> 토큰을 나타내는 0벡터)
# 두 단어의 평균 임베딩을 계산하고 그 결과에 2의 제곱근을 곱한다

tf.reduce_mean(another_example[0:1, :2], axis =1)*tf.sqrt(2.)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[3.535534 , 4.9497476, 2.1213202]], dtype=float32)>

In [75]:
# 두번째 리뷰 확인
# 이 리뷰는 하나의 단어만 가지고 있다 (두개의 패딩 토큰은 무시)

tf.reduce_mean(another_example[1:2, :1], axis = 1)*tf.sqrt(1.)

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[6., 0., 0.]], dtype=float32)>

In [77]:
# 최중모델 훈련
# BoW(Bag of Words) 층을 Embedding 층과 compute_mean_embedding을 호출하는 Lambda층으로 바꾼다

embedding_size = 20
model = keras.models.Sequential([
    text_vectorization,
    keras.layers.Embedding(input_dim=n_tokens,
                          output_dim=embedding_size,
                          mask_zero = True),
    keras.layers.Lambda(compute_mean_embedding),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid"),
])

### f. 모델을 훈련하고 얼마의 정확도가 나오는지 확인해보세요. 가능한 한 훈련 속도를 빠르게 하기 위해 파이프라인을 최적화해보세요.

In [78]:
model.compile(loss = "binary_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
model.fit(train_set, epochs = 5, validation_data = valid_set)

Epoch 1/5
782/782 [==============================] - 21s 15ms/step - loss: 0.5553 - accuracy: 0.7070 - val_loss: 0.5271 - val_accuracy: 0.7315
Epoch 2/5
782/782 [==============================] - 9s 9ms/step - loss: 0.4949 - accuracy: 0.7551 - val_loss: 0.5070 - val_accuracy: 0.7435
Epoch 3/5
782/782 [==============================] - 9s 9ms/step - loss: 0.4804 - accuracy: 0.7612 - val_loss: 0.5157 - val_accuracy: 0.7415
Epoch 4/5
782/782 [==============================] - 9s 10ms/step - loss: 0.4727 - accuracy: 0.7636 - val_loss: 0.5119 - val_accuracy: 0.7421
Epoch 5/5
782/782 [==============================] - 10s 9ms/step - loss: 0.4658 - accuracy: 0.7656 - val_loss: 0.5137 - val_accuracy: 0.7358


In [ ]:
# 임베딩을 사용해서 더 나이지지 않음
# 16장에서 이를 개선하는 내용이 나옴

### g. tfds.load("imdb_reviews")와 같이 TFDS를 사용해 동일한 데이터셋을 간단하게 적재해보세요.

In [81]:
# tensorflow_datasets 설치 먼저해야 함. 안그러면 에러 발생
!pip install tensorflow_datasets

     ---------------------------------------- 5.3/5.3 MB 26.1 MB/s eta 0:00:00
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
     ---------------------------------------- 102.1/102.1 kB ? eta 0:00:00
     -------------------------------------- 140.1/140.1 kB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 52.3/52.3 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 110.5/110.5 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached googleapis_common_protos-1.58.0-py2.py3-none-any.whl (223 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21554 sha256=54fc7d9cb44449e21707b3c4beaa94efef75d80629abde53d9dc6fa562e01563
  Stored in directory: c:\users\jaeho\appdata\local\pip\cache\wheels\9d\ad\15\e6d5c43a0f01b88ee5883bd249a18e09d72821e43b1c3e8187
Successfully built promise



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [82]:
import tensorflow_datasets as tfds

datasets = tfds.load(name = "imdb_reviews")
train_set, test_set = datasets["train"], datasets["test"]

0.1.0
Using C:\Users\jaeho\tensorflow_datasets\imdb_reviews\plain_text\1.0.0 instead.


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\jaeho\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete4URQ83\imdb_reviews-test.…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\jaeho\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete4URQ83\imdb_reviews-unsup…

Dataset imdb_reviews downloaded and prepared to C:\Users\jaeho\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [83]:
for example in train_set.take(1):
    print(example["text"])
    print(example["label"])

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)
